In [1]:
import cv2
import numpy as np
from keras.applications.resnet50 import ResNet50, preprocess_input
from sklearn.decomposition import PCA
import h5py, os

2024-04-02 10:59:29.881768: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-02 10:59:30.111622: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-02 10:59:30.999888: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
# Load pre-trained ResNet50 model without the classification layer
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

# Function to extract features from frames using ResNet50
def extract_features(frames):
    features_list = []
    for frame in frames:
        # Preprocess frame for ResNet50 input
        processed_frame = preprocess_input(frame)
        # Extract features from the frame
        features = base_model.predict(np.expand_dims(processed_frame, axis=0))
        features_list.append(features.flatten())
    return features_list

# Function to process video and generate embeddings
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frames = []

    # Read frames from the video
    for _ in range(frame_count):
        ret, frame = cap.read()
        if ret:
            frames.append(cv2.resize(frame, (224, 224)))  # Resize frame to match ResNet50 input size

    # Extract features from frames
    features = extract_features(frames)

    # Perform PCA for dimensionality reduction
    pca = PCA(n_components=512)  # Adjust the number of components as needed
    embeddings = pca.fit_transform(features)

    return embeddings

In [2]:
#  Process all videos in the folder
video_folder = "../../datasets/Dravidian Dataset/malayalam_data/videos"
output_folder = "../../datasets/Dravidian Dataset/malayalam_data/embeddings"
video_embeddings_path = os.path.join(output_folder, "video_embeddings.h5")

In [ ]:
# Create a dictionary to store video embeddings
video_embeddings = {}

for video_file in os.listdir(video_folder):
    if video_file.endswith(".mp4"):
        video_path = os.path.join(video_folder, video_file)
        embedding = process_video(video_path)
        video_name = os.path.splitext(video_file)[0]
        video_embeddings[video_name] = embedding

# Create an HDF5 file and write the data
with h5py.File(video_embeddings_path, 'w') as hf:
    for video_id, embedding in video_embeddings.items():
        hf.create_dataset(video_id, data=embedding)
        print(embedding.shape)

In [9]:
# Open the HDF5 file in read mode
with h5py.File(video_embeddings_path, 'r') as file:
    # List all the groups in the file
    print("Groups in the HDF5 file:")
    g = list(file.keys())
    print(g)
    for i in g:
        print(file[i].shape)

Groups in the HDF5 file:
['MAL_MSA_01', 'MAL_MSA_02', 'MAL_MSA_03', 'MAL_MSA_04', 'MAL_MSA_05', 'MAL_MSA_06', 'MAL_MSA_07', 'MAL_MSA_08', 'MAL_MSA_09', 'MAL_MSA_10', 'MAL_MSA_11', 'MAL_MSA_12', 'MAL_MSA_13', 'MAL_MSA_14', 'MAL_MSA_15', 'MAL_MSA_16', 'MAL_MSA_18', 'MAL_MSA_19', 'MAL_MSA_20', 'MAL_MSA_21', 'MAL_MSA_22', 'MAL_MSA_23', 'MAL_MSA_24', 'MAL_MSA_25', 'MAL_MSA_26', 'MAL_MSA_27', 'MAL_MSA_28', 'MAL_MSA_29', 'MAL_MSA_30', 'MAL_MSA_31', 'MAL_MSA_32', 'MAL_MSA_33', 'MAL_MSA_34', 'MAL_MSA_35', 'MAL_MSA_36', 'MAL_MSA_37', 'MAL_MSA_38', 'MAL_MSA_39', 'MAL_MSA_40', 'MAL_MSA_41', 'MAL_MSA_42', 'MAL_MSA_43', 'MAL_MSA_44', 'MAL_MSA_45', 'MAL_MSA_46', 'MAL_MSA_47', 'MAL_MSA_48', 'MAL_MSA_49', 'MAL_MSA_50', 'MAL_MSA_51', 'MAL_MSA_52', 'MAL_MSA_53', 'MAL_MSA_54', 'MAL_MSA_55', 'MAL_MSA_56', 'MAL_MSA_57', 'MAL_MSA_58', 'MAL_MSA_59', 'MAL_MSA_60', 'MAL_MSA_61', 'MAL_MSA_62', 'MAL_MSA_63', 'MAL_MSA_64', 'MAL_MSA_65', 'MAL_MSA_66', 'MAL_MSA_67', 'MAL_MSA_68', 'MAL_MSA_69', 'MAL_MSA_70']
(2745, 5